# Week 2 Exercise: Build a Multimodal AI Technical Assistant
## Task Objective:
This week, you will upgrade the technical question-answering tool you built in Week 1 into a full-featured prototype. The goal is to apply all the concepts from Week 2 to create a more robust and interactive AI assistant.

Core Requirements
Your prototype must include the following features:

Gradio UI: Build a user-friendly interface for your assistant.

Streaming Responses: Ensure the model's answers are streamed back to the user in real-time.

System Prompt: Use a system prompt to define the AI's expertise and personality as a technical assistant.

Model Switching: Implement a feature that allows the user to switch between at least two different models (e.g., an OpenAI model and Llama).

Bonus: Demonstrate the use of a tool within your application for extra credit.
 

In [65]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [66]:
# Constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [67]:
# set up environment
load_dotenv(override=True)
api_key=os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [68]:
# initializing  models

openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [69]:
# here is the question; type over this to ask something new
system_prompt = """
You are a technical assistant. You are to expect technical questions. You are to respond with detailed explanations to the technical question. You are to respond in Nigerian Pidgin English
"""

question = """
Please explain what tokens are in LLM
"""


messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

In [70]:
def chat(message, history, model_choice):
    # history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = (
        [{"role": "system", "content": system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )
    if model_choice == "openai":
        try:
            stream = openai.chat.completions.create(
                model=MODEL_GPT, messages=messages, stream=True
            )
            
            response = ""
            for chunk in stream:
                response += chunk.choices[0].delta.content or ""
                yield response
        except Exception as e:
            yield f"Error with OpenAI API: {e}"

    elif model_choice == "llama":
        try:
            stream = ollama.chat.completions.create(
                model=MODEL_LLAMA, messages=messages, stream=True
            )
            
            response = ""
            for chunk in stream:
                response += chunk.choices[0].delta.content or ""
                yield response
        except Exception as e:
            yield f"Error with OpenAI API: {e}"

In [71]:
model_selector = gr.Radio(
    ["openai", "llama"], 
    label="Choose Model", 
    value="openai" 
)

In [72]:
chatbot = gr.ChatInterface(fn=chat, type="messages", additional_inputs=[model_selector])

In [73]:
chatbot.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
